In [ ]:
import pandas as pd
import numpy as np
df2 = pd.read_excel('empdata.xlsx', sheet_name=1, header=1)
df2.drop(index=0, inplace= True)
df2.rename(columns={'Check-in Count':'User ID'}, inplace=True)
df3 = pd.read_excel('empdata.xlsx', sheet_name=2,header = [0,1])
midyrcolumn = ['Jan','Feb','Mar','Apr','May','Jun' ]
df2['midtotalgit'] = df2[midyrcolumn].sum(axis = 1)
df2['totalgit'] = df2.sum(axis=1)
usefulcols = ('User ID','midtotalgit','totalgit')
usefuldf2 = df2.filter(usefulcols, axis=1)
gitdata = df2.drop(columns = ['midtotalgit','totalgit'], inplace = False)

df1 = pd.read_excel('empdata.xlsx', sheet_name=0, header=1)
l=[i for i in range(1,len(df1['2020 mid year rating'].unique())+1)]
dic = dict(zip(df1['2020 mid year rating'].unique(),l))
df1['midrating'] = df1['2020 mid year rating'].replace(dic)
df1['endrating'] = df1['2020 year end rating'].replace(dic)
finaldf = pd.merge(df1,usefuldf2)


def checkcategorybias(rating,git,category):
    mean = finaldf[finaldf['Job Category']==category]['totalgit'].mean()
    if rating==1:
        if(git<mean):
            return "Unbiased"
        else:
            return "Biased"
    elif rating == 2:
        if(git<=mean):
            return "Unbiased"
        else:
            return "Biased"
    elif rating ==3:
        if(git>=mean):
            return "Unbiased"
        else:
            return "Biased"
    elif rating ==4:
        if(git>mean):
            return "Unbiased"
        else:
            return "Biased"
    elif rating ==5:
        if(git>=mean):
            return "Unbiased"
        else:
            return "Biased"

def checkoverallbias(rating,git):
    mean = finaldf['totalgit'].mean()
    if rating==1:
        if(git<mean):
            return "Unbiased"
        else:
            return "Biased"
    elif rating == 2:
        if(git<=mean):
            return "Unbiased"
        else:
            return "Biased"
    elif rating ==3:
        if(git>=mean):
            return "Unbiased"
        else:
            return "Biased"
    elif rating ==4:
        if(git>mean):
            return "Unbiased"
        else:
            return "Biased"
    elif rating ==5:
        if(git>=mean):
            return "Unbiased"
        else:
            return "Biased"
def linkgenerator(string):
    return "http://127.0.0.1:5000/profile?uid="+string

finaldf['Category Bias']=finaldf.apply(lambda x: checkcategorybias(x.endrating,x.totalgit,x['Job Category']), axis = 1)
finaldf['Overall Bias']=finaldf.apply(lambda x: checkoverallbias(x.endrating,x.totalgit), axis = 1)
finaldf['Link']=finaldf['User ID'].apply(linkgenerator)
columns = ['Link','Employee ID', 'User ID', 'Name', 'Line Manager', 'Date of Joining',
       'Job Category', '2020 mid year rating', '2020 year end rating',
       'midrating', 'endrating', 'midtotalgit', 'totalgit', 'Category Bias',
       'Overall Bias']
finaldf=finaldf[columns]
finaldf = finaldf.sort_values(by = 'endrating', ascending = True)

rating_git = []
for i in range(1,6):
    rating_git.append(finaldf[finaldf['midrating'] ==i]['totalgit'].sum())
jobs = finaldf['Job Category'].unique()
categdic = {}
for i in jobs:
    categdic[i] = finaldf[finaldf['Job Category']==i]

#### OVERALL PAGE VARIABLES
midavg = []
endavg = []
for i in jobs:
    midavg.append(categdic[i]['midrating'].astype(float).mean())
for i in jobs:
    endavg.append(categdic[i]['endrating'].astype(float).mean())
    
midsum = []
endsum = []
for i in jobs:
    midsum.append(categdic[i]['midrating'].astype(float).sum())
for i in jobs:
    endsum.append(categdic[i]['endrating'].astype(float).sum())
categgitavg=[]
ratinggitavg = []
for i in jobs:
    categgitavg.append(categdic[i]['totalgit'].mean())
for i in range(1,6):
    ratinggitavg.append(finaldf.loc[finaldf['endrating']==i]['totalgit'].mean())

#### CATEGORY PAGE VARIABLES
analysts =categdic['Analyst']
analystratingwise_count = []
for i in range(1,6):
    analystratingwise_count.append(len(analysts.loc[analysts['endrating']==i]))
analystratingwise_gitavg = []
for i in range(1,6):
    analystratingwise_gitavg.append(analysts.loc[analysts['endrating']==i]['totalgit'].mean())
analystratingwise_gitavg = [0 if x != x else x for x in analystratingwise_gitavg]
analystdf = analysts.drop(columns=['User ID','Line Manager','midrating','endrating','midtotalgit','Job Category','Date of Joining'])

associates =categdic['Associate']
associateratingwise_count = []
for i in range(1,6):
    associateratingwise_count.append(len(associates.loc[associates['endrating']==i]))
associateratingwise_gitavg = []
for i in range(1,6):
    associateratingwise_gitavg.append(associates.loc[associates['endrating']==i]['totalgit'].mean())
associateratingwise_gitavg = [0 if x != x else x for x in associateratingwise_gitavg]
associatedf = associates.drop(columns=['User ID','Line Manager','midrating','endrating','midtotalgit','Job Category'])

SEs =categdic['Software Engineer']
SEratingwise_count = []
for i in range(1,6):
    SEratingwise_count.append(len(SEs.loc[SEs['endrating']==i]))
SEratingwise_gitavg = []
for i in range(1,6):
    SEratingwise_gitavg.append(SEs.loc[SEs['endrating']==i]['totalgit'].mean())
SEratingwise_gitavg = [0 if x != x else x for x in SEratingwise_gitavg]
SEdf = SEs.drop(columns=['User ID','Line Manager','midrating','endrating','midtotalgit','Job Category'])

SSEs =categdic['Software Senior Engineer']
SSEratingwise_count = []
for i in range(1,6):
    SSEratingwise_count.append(len(SSEs.loc[SSEs['endrating']==i]))
SSEratingwise_gitavg = []
for i in range(1,6):
    SSEratingwise_gitavg.append(SSEs.loc[SSEs['endrating']==i]['totalgit'].mean())
SSEratingwise_gitavg = [0 if x != x else x for x in SSEratingwise_gitavg]
SSEdf = SSEs.drop(columns=['User ID','Line Manager','midrating','endrating','midtotalgit','Job Category'])

SESs =categdic['Software Engineer Specialty']
SESratingwise_count = []
for i in range(1,6):
    SESratingwise_count.append(len(SESs.loc[SESs['endrating']==i]))
SESratingwise_gitavg = []
for i in range(1,6):
    SESratingwise_gitavg.append(SESs.loc[SESs['endrating']==i]['totalgit'].mean())
SESratingwise_gitavg = [0 if x != x else x for x in SESratingwise_gitavg]
SESdf = SESs.drop(columns=['User ID','Line Manager','midrating','endrating','midtotalgit','Job Category'])

SESSs =categdic['Software Senior Engineer Specialty']
SESSratingwise_count = []
for i in range(1,6):
    SESSratingwise_count.append(len(SESSs.loc[SESSs['endrating']==i]))
SESSratingwise_gitavg = []
for i in range(1,6):
    SESSratingwise_gitavg.append(SESSs.loc[SESSs['endrating']==i]['totalgit'].mean())
SESSratingwise_gitavg = [0 if x != x else x for x in SESSratingwise_gitavg]
SESSdf = SESSs.drop(columns=['User ID','Line Manager','midrating','endrating','midtotalgit','Job Category'])

categgitaverage = []
for j in jobs:
        categgitaverage.append(categdic[j].mean())

from flask import Flask, render_template, request
import json
app = Flask(__name__, static_url_path='/static')
@app.route('/', methods = ['GET','POST'])
def upload_file():
    return render_template('upload.html')
@app.route('/uploader', methods = ['GET','POST'])
def uploader():
    try:    
        f = request.files['file']
        f.save('empdata.xlsx')
    finally:
        jobcatjson = json.dumps(list(jobs))
        midyearavgjson = json.dumps(midavg)
        endyearavgjson = json.dumps(endavg)
        midyearsumjson = json.dumps(midsum)
        endyearsumjson = json.dumps(endsum)
        categgitavgjson = json.dumps(categgitavg)
        ratinggitavgjson = json.dumps(ratinggitavg)
        ratinglabelsjson =  json.dumps(list(df1['2020 mid year rating'].unique()))
        analystratingwise_gitavgjson = json.dumps(analystratingwise_gitavg)
        analystratingwise_countjson = json.dumps(analystratingwise_count)
        associateratingwise_gitavgjson = json.dumps(associateratingwise_gitavg)
        associateratingwise_countjson = json.dumps(associateratingwise_count)
        SEratingwise_gitavgjson = json.dumps(SEratingwise_gitavg)
        SEratingwise_countjson = json.dumps(SEratingwise_count)
        SSEratingwise_gitavgjson = json.dumps(SSEratingwise_gitavg)
        SSEratingwise_countjson = json.dumps(SSEratingwise_count)
        SESratingwise_gitavgjson = json.dumps(SESratingwise_gitavg)
        SESratingwise_countjson = json.dumps(SESratingwise_count)
        SESSratingwise_gitavgjson = json.dumps(SESSratingwise_gitavg)
        SESSratingwise_countjson = json.dumps(SESSratingwise_count)
        return render_template('data.html', jobs = jobcatjson, midavg = midyearavgjson, endavg =endyearavgjson ,midsum = midyearsumjson, endsum =endyearsumjson, categgit =  categgitavgjson ,ratinggit = ratinggitavgjson, ratinglabels = ratinglabelsjson, 
                               analyst_rwg = analystratingwise_gitavgjson,analyst_rwc = analystratingwise_countjson, analysttables=[analystdf.to_html(classes='sortable', header="true", table_id="grades" , index=False, render_links = True)], 
                               associate_rwg = associateratingwise_gitavgjson,associate_rwc = associateratingwise_countjson, associatetables=[associatedf.to_html(classes='sortable', header="true", table_id="grades" , index=False, render_links = True)], 
                               SE_rwg = SEratingwise_gitavgjson,SE_rwc = SEratingwise_countjson, SEtables=[SEdf.to_html(classes='sortable', header="true", table_id="grades" , index=False, render_links = True)], 
                               SSE_rwg = SSEratingwise_gitavgjson,SSE_rwc = SSEratingwise_countjson, SSEtables=[SSEdf.to_html(classes='sortable', header="true", table_id="grades" , index=False, render_links = True)],
                              SES_rwg = SESratingwise_gitavgjson,SES_rwc = SESratingwise_countjson, SEStables=[SESdf.to_html(classes='sortable', header="true", table_id="grades" , index=False, render_links = True)],
                              SESS_rwg = SESSratingwise_gitavgjson,SESS_rwc = SESSratingwise_countjson, SESStables=[SESSdf.to_html(classes='sortable', header="true", table_id="grades" , index=False, render_links = True)],)
@app.route('/profile', methods = ['GET','POST'])
def profile():
    uid = request.args.get('uid')
    row = finaldf.loc[finaldf['User ID']==uid]
    j = [int(row['totalgit'][row.index[0]]) ]* len(finaldf['User ID'])
    l = [int(row['totalgit'][row.index[0]]) ]* len(categdic[row['Job Category'][row.index[0]]]['totalgit'].to_list())
    k = [int(row['endrating'][row.index[0]])]* len(categdic[row['Job Category'][row.index[0]]]['endrating'].to_list())
    m = [int(row['totalgit'][row.index[0]])]* len(jobs)
    categk = json.dumps(k)
    categl = json.dumps(l)
    gitj = json.dumps(j)
    gitjobs = json.dumps(m)
    userjson = json.dumps(categdic[row['Job Category'][row.index[0]]]['User ID'].to_list())
    categitjson = json.dumps(categdic[row['Job Category'][row.index[0]]]['totalgit'].to_list())
    endjson = json.dumps(categdic[row['Job Category'][row.index[0]]]['endrating'].to_list())
    vals = df3[df3['Unnamed: 0_level_0']['User ID']==uid].drop('Unnamed: 0_level_0', axis = 1).values[0]
    jirac = vals[::2]
    jirac = jirac.tolist()
    storyp = vals[1::2]
    storyp = storyp.tolist()
    jirajson = json.dumps(jirac)
    storyjson = json.dumps(storyp)
    alluserjson = json.dumps(finaldf['User ID'].to_list())
    allgitjson = json.dumps(finaldf['totalgit'].to_list())
    gitlist = gitdata[gitdata['User ID']==uid].drop('User ID', axis=1).values[0].tolist()
    gitjson = json.dumps(gitlist)
    categgitaveragejson = json.dumps(categgitaverage[int(row['endrating'][row.index[0]])-1].tolist())
    return render_template('profile.html', 
                           Emp_id=row['Employee ID'][row.index[0]],
                           uid = row['User ID'][row.index[0]],
                           name = row['Name'][row.index[0]],
                           lm = row['Line Manager'][row.index[0]],
                           doj = row['Date of Joining'][row.index[0]],
                           cat= row['Job Category'][row.index[0]],
                           catbias = row['Category Bias'][row.index[0]],
                           ovbias = row['Overall Bias'][row.index[0]],
                           rating = row['2020 year end rating'][row.index[0]],
                           users = userjson,
                           git = categitjson,
                           end = endjson,
                           usergit= categl,
                           userend = categk,
                           jira = jirajson,
                           sp = storyjson,
                           gitd = gitjson,
                           allusers = alluserjson,
                           allgit  = allgitjson,
                           empgit = gitj,
                           gitavg = categgitaveragejson,
                           userjobs = gitjobs
                          )
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Apr/2021 23:52:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 23:52:40] "GET /bourbon HTTP/1.1" 404 -
127.0.0.1 - - [25/Apr/2021 23:52:45] "POST /uploader HTTP/1.1" 200 -
C:\Users\Keshav\anaconda3\lib\site-packages\pandas\core\generic.py:3887: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
127.0.0.1 - - [25/Apr/2021 23:52:48] "GET /profile?uid=U2041 HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 23:55:27] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2021 23:55:28] "GET /bourbon HTTP/1.1" 404 -
127.0.0.1 - - [25/Apr/2021 23:55:34] "POST /uploader HTTP/1.1" 200 -
C:\Users\Keshav\anaconda3\lib\site-packages\pandas\core\generic.py:3887: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, lev